In [3]:
import pandas as pd
from subprocess import call,run
import matplotlib.pyplot as plt
import os
import numpy as np
import seaborn as sns
import pickle
import re
import itertools

In [41]:
def load_and_filter_qtl(qtl_file,qtl_type):
    if qtl_type == "stQTL":
        cols_to_read = ["chrom","strand","snp_pos_1base","rsID","pvalue"]
        qtl_data = pd.read_csv(qtl_file, usecols=cols_to_read)
        qtl_filtered = qtl_data[qtl_data['pvalue'] < 0.05]
        qtl_filtered['s'] = qtl_filtered['snp_pos_1base']-1
        qtl_filtered['e'] = qtl_filtered['snp_pos_1base']+1
        return qtl_filtered[['chrom', 's', 'e', 'rsID', 'pvalue', 'strand']]
    else:
        cols_to_read = ["chrom","strand","snp_pos_1base","rsID","BayesFactor"]
        qtl_data = pd.read_csv(qtl_file, usecols=cols_to_read)
        qtl_filtered = qtl_data[qtl_data['BayesFactor'] > 3]
        qtl_filtered['s'] = qtl_filtered['snp_pos_1base']-1
        qtl_filtered['e'] = qtl_filtered['snp_pos_1base']+1
        return qtl_filtered[['chrom', 's', 'e', 'rsID', 'BayesFactor', 'strand']] # chrom   start   end   name   score   strand


In [42]:
def get_lead_qtl(qtl_genes_bed,qtl_type):
    qtl_genes = pd.read_csv(qtl_genes_bed, sep='\t', header=None,usecols=[0,1,3,4,9])
    if qtl_type == "stQTL":
        qtl_genes.columns = ['chrom', 'snp_pos', 'rsID', 'pvalue', 'geneID']
        qtl_genes['snp_pos'] = qtl_genes['snp_pos']+1
        lead_qtls = qtl_genes.loc[qtl_genes.groupby('geneID')['pvalue'].idxmin()]
    else:
        qtl_genes.columns = ['chrom', 'snp_pos', 'rsID', 'BF', 'geneID']
        qtl_genes['snp_pos'] = qtl_genes['snp_pos']+1
        # 对每个基因（geneID）分组，保留BayesFactor（BF）最大的那个QTL
        lead_qtls = qtl_genes.loc[qtl_genes.groupby('geneID')['BF'].idxmax()]
    return lead_qtls

In [43]:
def analyze_leadQTL_distance(qtl_file_1, qtl_file_2, qtl1, qtl2, gene_bed_file, chrom_size_file, work_dir, bim_dict, bfile_path):
    # 设置工作目录
    os.chdir(work_dir)

    # Step 1: 读取并筛选QTL summary数据
    qtl_filtered_1 = load_and_filter_qtl(qtl_file_1,qtl1)
    qtl_filtered_2 = load_and_filter_qtl(qtl_file_2,qtl2)

    # 保存为BED文件
    qtl_filtered_1.to_csv('qtl_1.bed', sep='\t', header=False, index=False)
    qtl_filtered_2.to_csv('qtl_2.bed', sep='\t', header=False, index=False)

    # Step 2: 排序BED文件
    call(f"sort -t, -k1,1V -k2,2n qtl_1.bed > qtl_sorted_1.bed",shell=True)
    call(f"sort -t, -k1,1V -k2,2n qtl_2.bed > qtl_sorted_2.bed",shell=True)

    # Step 3: 使用bedtools intersect找出QTL所在的基因
    call(f"bedtools intersect -a qtl_sorted_1.bed -b {gene_bed_file} -g {chrom_size_file} -wa -wb -s -sorted > qtl_1_gene.bed",shell=True)
    call(f"bedtools intersect -a qtl_sorted_2.bed -b {gene_bed_file} -g {chrom_size_file} -wa -wb -s -sorted > qtl_2_gene.bed",shell=True)                  

    # Step 4: 读取QTL和基因信息并找到每个基因的lead QTL
    lead_qtls_1 = get_lead_qtl('qtl_1_gene.bed',qtl1)
    lead_qtls_2 = get_lead_qtl('qtl_2_gene.bed',qtl2)
    lead_qtls_1 = lead_qtls_1.rename(columns={'rsID': 'rsID_1', 'snp_pos': 'snp_pos_1'})
    lead_qtls_2 = lead_qtls_2.rename(columns={'rsID': 'rsID_2', 'snp_pos': 'snp_pos_2'})
    merged_qtls = pd.merge(lead_qtls_1, lead_qtls_2[['geneID', 'rsID_2', 'snp_pos_2']], on='geneID', how='inner')

    # Step 5: 计算两个lead QTL之间的距离
    merged_qtls['dis'] = abs(merged_qtls['snp_pos_2'] - merged_qtls['snp_pos_1'])

    # 创建新的列 dis_group 根据预设的区间来划分 dis
    bins = [0, 10, 100, 1000, 10000, 100000]  # 设置区间
    labels = ['0', '10', '100', '1000', '10000']
    merged_qtls['dis_group'] = pd.cut(merged_qtls['dis'], bins=bins, labels=labels, right=False)

    # 设置绘图样式
    sns.set(style="whitegrid")
    # 绘制柱状图
    plt.figure(figsize=(10, 6))
    sns.countplot(data=merged_qtls, x='dis_group', color='skyblue', edgecolor='black',width=0.4)
    # 添加标题和标签
    # plt.title('Distribution of Lead SNP Distance by Range', fontsize=16)
    plt.xlabel(f'Distance Range Between {qtl1} and {qtl2} (bp)', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    # 调整布局
    plt.tight_layout()
    # 保存图表为PDF
    plt.savefig(f'{qtl1}_{qtl2}_leaddis.pdf')

    # 检查是否包含 "rs"
    mask_rsID_1 = merged_qtls['rsID_1'].str.contains('rs', na=False)
    mask_rsID_2 = merged_qtls['rsID_2'].str.contains('rs', na=False)

    # 将 snp_pos_1 和 snp_pos_2 列转换为字符串类型
    merged_qtls['snp_pos_1'] = merged_qtls['snp_pos_1'].astype(str)
    merged_qtls['snp_pos_2'] = merged_qtls['snp_pos_2'].astype(str)

    # 使用位置信息替换不包含 "rs" 的值
    merged_qtls.loc[~mask_rsID_1, 'rsID_1'] = merged_qtls.loc[~mask_rsID_1, 'chrom'].str.replace('ch', '') + "_" + merged_qtls.loc[~mask_rsID_1, 'snp_pos_1'].str[2:]
    merged_qtls.loc[~mask_rsID_2, 'rsID_2'] = merged_qtls.loc[~mask_rsID_2, 'chrom'].str.replace('ch', '') + "_" + merged_qtls.loc[~mask_rsID_2, 'snp_pos_2'].str[2:]
    # 替换rsID为bim中的SNP name
    merged_qtls['rsID_1'] = merged_qtls['rsID_1'].map(bim_dict)
    merged_qtls['rsID_2'] = merged_qtls['rsID_2'].map(bim_dict)
    merged_qtls = merged_qtls.dropna(subset=['rsID_1', 'rsID_2'])
    merged_qtls = merged_qtls.reset_index(drop=True)
    res_df = add_ld_to_dataframe(merged_qtls, bfile_path)
    res_df.to_csv(f'{qtl1}_{qtl2}_LD.csv', index=False)
    


In [1]:
gene_bed_file = '/mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/GeneFunctionalPathways/gff_sorted.bed'
chrom_size_file = '/mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/GeneFunctionalPathways/hg19.chrom.sizes.sorted'
work_dir = '/mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/leadSNP_dis_LD'
bfile_path = '/mnt/hpc/home/xuxinran/huvec_genotype/huvec_imputed'


qtl_files = [
    '/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/Iqtl/nano_merge_I_summary.csv',
    '/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/puqtl/nano_merge_promoter_summary.csv',
    '/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/m6Aqtl/nano_merge_m6A_summary.csv',
    '/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/pseUqtl/nano_merge_pseU_summary.csv',
    '/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/m5Cqtl/nano_merge_m5C_summary.csv',
    '/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/stqtl/nano_merge_stability_summary.csv',
    '/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/3aqtl/nano_merge_APA_summary.csv',
    '/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.8.1/irqtl/nano_merge_isoform_summary.csv'
]
qtl_names = [
    'inosine-QTL',
    'puQTL',
    'm6A-QTL',
    'pseU-QTL',
    'm5C-QTL',
    'stQTL',
    '3aQTL',
    'irQTL'
]

In [4]:
# 生成sh文件
res = []

for i, j in itertools.combinations(range(len(qtl_names)), 2):
        qtl_file_1 = qtl_files[i]
        qtl_file_2 = qtl_files[j]
        qtl1 = qtl_names[i]
        qtl2 = qtl_names[j]
        c = f'python /mnt/hpc/home/xuxinran/code/methy/nanopore_RNA/downsteam/leadSNP_dis_LD.py -c {chrom_size_file} -d {work_dir} --geno {bfile_path} --gff {gene_bed_file} -b bim_dict.pkl --qf1 {qtl_file_1} --qf2 {qtl_file_2} --q1 {qtl1} --q2 {qtl2}'
        res.append(c)


In [8]:
# 保存res为txt文件
with open('/mnt/hpc/home/xuxinran/DirectSeq/8_downsteam/leadSNP_dis_LD/run.sh', 'w') as f:
    for item in res:
        f.write("%s\n" % item)

In [15]:
def analyze_all_qtl_pairs(qtl_files, qtl_names, gene_bed_file, chrom_size_file, work_dir, bim_dict, bfile_path):
    """
    对所有QTL进行两两分析。

    Args:
        qtl_files: 包含所有QTL文件路径的列表。
        qtl_names: 包含所有QTL名称的列表。
        gene_bed_file: 基因bed文件路径。
        chrom_size_file: 染色体大小文件路径。
        work_dir: 工作目录。
        bim_dict: bim文件字典。
        bfile_path: bfile文件路径。
    """

    # 使用itertools.combinations生成所有可能的QTL对
    for i, j in itertools.combinations(range(len(qtl_names)), 2):
        qtl_file_1 = qtl_files[i]
        qtl_file_2 = qtl_files[j]
        qtl1 = qtl_names[i]
        qtl2 = qtl_names[j]
        print(f'Analyzing {qtl1} and {qtl2}...')
        analyze_leadQTL_distance(qtl_file_1, qtl_file_2, qtl1, qtl2, gene_bed_file, chrom_size_file, work_dir, bim_dict, bfile_path)


In [ ]:
analyze_all_qtl_pairs(qtl_files, qtl_names, gene_bed_file, chrom_size_file, work_dir, bim_dict, bfile_path)

In [ ]:
## 矫正rsID为bim中的SNP name的文件

bim_file = "/mnt/hpc/home/xuxinran/huvec_genotype/huvec_imputed.bim"
bim_df = pd.read_csv(bim_file, sep='\t', header=None, usecols=[1])
bim_df.columns = ['SNP_name']
# 使用str.contains()方法判断是否包含":"或"_"
mask_colon = bim_df['SNP_name'].str.contains(":")
mask_underscore = bim_df['SNP_name'].str.contains("_")
# 使用str.split()方法分割字符串
bim_df.loc[mask_colon, 'rsID'] = bim_df.loc[mask_colon, 'SNP_name'].str.split(":", n=1, expand=True)[0]
bim_df.loc[mask_underscore, 'rsID'] = bim_df.loc[mask_underscore, 'SNP_name'].str.split("_", n=2, expand=True).apply(lambda x: "_".join(x[:2]), axis=1)
# 处理不包含":"或"_"的SNP_name
bim_df.loc[~(mask_colon | mask_underscore), 'rsID'] = bim_df.loc[~(mask_colon | mask_underscore), 'SNP_name']

bim_dict = pd.Series(bim_df['SNP_name'].values, index=bim_df['rsID']).to_dict()

# 将字典保存到指定路径
with open('bim_dict.pkl', 'wb') as f:
    pickle.dump(bim_dict, f)

# 读取
with open('bim_dict.pkl', 'rb') as f:
    bim_dict = pickle.load(f)

In [17]:
def run_plink_ld(bfile_path, rs1, rs2):
    if rs1 == rs2:
        return 1, 1
    else:
        command = [
                'plink',
                '--bfile', bfile_path,
                '--ld', rs1, rs2
            ]
        res = run(command, capture_output=True, text=True, check=True)
        filtered_ld_list = extract_r_sq_d(res.stdout)
        r2 ,d = filtered_ld_list[0][0], filtered_ld_list[0][1]
        return r2 ,d



In [18]:
def extract_r_sq_d(plink_output):
    r_sq_d_list = []
    pattern = r"R-sq = (.*)\s+D' = (.*)\n\n"  # 匹配R-sq和D'的正则表达式
    matches = re.findall(pattern, plink_output)
    for match in matches:
        r_sq = float(match[0])
        d = float(match[1])
        r_sq_d_list.append((r_sq, d))
    filtered_ld_list = filter_ld_results(r_sq_d_list)
    return filtered_ld_list


In [19]:
def filter_ld_results(r_sq_d_list, low_threshold=0.2, high_threshold=0.8):

    if len(r_sq_d_list) == 1:  # 如果只有一个结果，则直接保留
        return r_sq_d_list

    filtered_ld_list = []
    for r_sq, d in r_sq_d_list:
        if r_sq < low_threshold and d < low_threshold:
            continue  # 剔除R-sq和D'都很小的结果
        elif r_sq > high_threshold and d > high_threshold:
            filtered_ld_list.append((r_sq, d))  # 保留R-sq和D'都很大的结果
        elif (r_sq > low_threshold and d < low_threshold) or (r_sq < low_threshold and d > low_threshold):
            continue  # 剔除R-sq和D'一个大一个小的结果
        else:
            filtered_ld_list.append((r_sq, d)) # 保留其他结果

    if len(filtered_ld_list) > 0:
        return filtered_ld_list
    else:
        # 如果所有结果都被剔除，则选择一个相对合理的结果
        reasonable_result = None
        for r_sq, d in r_sq_d_list:
            if reasonable_result is None:
                reasonable_result = (r_sq, d)
            elif abs(r_sq - d) < abs(reasonable_result[0] - reasonable_result[1]):  # 选择R-sq和D'差异最小的结果
                reasonable_result = (r_sq, d)
        return [reasonable_result]


In [20]:
def add_ld_to_dataframe(res1_df, bfile_path):

    r2_list = []
    d_list = []

    for index, row in res1_df.iterrows():
        rs1 = row['rsID_1']
        rs2 = row['rsID_2']
        r2, d = run_plink_ld(bfile_path, rs1, rs2)
        r2_list.append(r2)
        d_list.append(d)

    res1_df['R_sq'] = r2_list
    res1_df['D'] = d_list

    return res1_df
